Backtest for Crossover Trading Strategy  
Two EMAs T

In [14]:
#EMA cross over strategy back test
import pandas as pd
import yfinance as yf

def backtest_ema_strategy(ticker, start_date, end_date, short_span=9, long_span=20, stop_loss=None):
    # Download historical data
    data = yf.download(ticker, start=start_date, end=end_date, group_by='column')
    data.columns = ['_'.join(filter(None, col)).strip() for col in data.columns]
    data.rename(columns=lambda x: x.replace(f"_{ticker}", ""), inplace=True)
    data.reset_index(inplace=True)

    # Ensure 'Close' exists
    if 'Close' not in data.columns:
        print("Error: 'Close' column not found!")
        return

    # Calculate EMAs
    data['Short_EMA'] = data['Close'].ewm(span=short_span, adjust=False).mean()
    data['Long_EMA'] = data['Close'].ewm(span=long_span, adjust=False).mean()
    data.dropna(subset=['Short_EMA', 'Long_EMA'], inplace=True)
    
    # Debugging: Check data after EMA calculation
    print(f"Data shape after EMA calculation: {data.shape}")
    if data.empty:
        print("Error: Data is empty after EMA calculation. Adjust EMA spans or data range.")
        return

    # Generate signals with a trend filter
    data['Signal'] = (data['Short_EMA'] > data['Long_EMA']).astype(int)
    data['Shifted_Signal'] = data['Signal'].shift(1)

    # Drop rows with NaN in 'Shifted_Signal'
    data.dropna(subset=['Shifted_Signal'], inplace=True)
    print(f"Data shape after dropping NaNs in Shifted_Signal: {data.shape}")
    if data.empty:
        print("Error: Data is empty after dropping NaNs in Shifted_Signal.")
        return

    # Calculate daily returns
    data['returns'] = data['Close'].pct_change()
    data.dropna(subset=['returns'], inplace=True)
    print(f"Data shape after dropping NaNs in returns: {data.shape}")
    if data.empty:
        print("Error: Data is empty after dropping NaNs in returns.")
        return

    # Calculate Strategy Returns
    data['Strategy_Return'] = data['Shifted_Signal'] * data['returns']

    # Apply stop-loss if specified
    if stop_loss:
        data['Max_Drawdown'] = data['Close'].expanding().max()  # Track maximum price
        data['Drawdown'] = (data['Close'] - data['Max_Drawdown']) / data['Max_Drawdown']
        data.loc[data['Drawdown'] < -stop_loss, 'Strategy_Return'] = 0

    # Ensure there is data left to calculate returns
    if data.empty:
        print("Error: No data available for return calculations.")
        return

    # Calculate total returns
    total_strategy_return = (1 + data['Strategy_Return']).prod() - 1
    total_benchmark_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1

    # Signal Analysis
    buy_signal_count = data['Signal'].sum()
    print(f"\nNumber of Buy Signals: {buy_signal_count}")
    print(f"Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%")

    # Build report
    report = f"""
    EMA Strategy Backtest Report for {ticker}
    ==================================
    Period: {start_date} to {end_date}
    EMA Strategy Parameters:
    - Short EMA Span: {short_span} days
    - Long EMA Span: {long_span} days
    - Stop Loss: {stop_loss * 100:.2f}% (if applicable)

    Performance:
    -----------------
    Total Strategy Return: {total_strategy_return * 100:.2f}%
    Total Buy-and-Hold Return (Benchmark): {total_benchmark_return * 100:.2f}%

    Signal Analysis:
    -----------------
    Number of Buy Signals: {buy_signal_count}
    Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%
    """
    print(report)

    # Optional: Return the data for further analysis
    return data

# Example usage
ticker = "TSLA"
data = backtest_ema_strategy(ticker, start_date="2024-07-01", end_date="2025-01-16", short_span=9, long_span=15, stop_loss=0.05)


[*********************100%***********************]  1 of 1 completed

Data shape after EMA calculation: (137, 8)
Data shape after dropping NaNs in Shifted_Signal: (136, 10)
Data shape after dropping NaNs in returns: (135, 11)

Number of Buy Signals: 94
Percentage of Days in Position: 69.63%

    EMA Strategy Backtest Report for TSLA
    Period: 2024-07-01 to 2025-01-16
    EMA Strategy Parameters:
    - Short EMA Span: 9 days
    - Long EMA Span: 15 days
    - Stop Loss: 5.00% (if applicable)

    Performance:
    -----------------
    Total Strategy Return: 177.52%
    Total Buy-and-Hold Return (Benchmark): 73.80%

    Signal Analysis:
    -----------------
    Number of Buy Signals: 94
    Percentage of Days in Position: 69.63%
    


In [13]:
#SMA
import pandas as pd
import yfinance as yf

def backtest_sma_strategy(ticker, start_date, end_date, short_window=5, long_window=15, stop_loss=None):
    # Download historical data
    data = yf.download(ticker, start=start_date, end=end_date, group_by='column')
    data.columns = ['_'.join(filter(None, col)).strip() for col in data.columns]
    data.rename(columns=lambda x: x.replace(f"_{ticker}", ""), inplace=True)
    data.reset_index(inplace=True)

    # Ensure 'Close' exists
    if 'Close' not in data.columns:
        print("Error: 'Close' column not found!")
        return

    # Calculate SMAs
    data['Short_SMA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_SMA'] = data['Close'].rolling(window=long_window).mean()
    data.dropna(subset=['Short_SMA', 'Long_SMA'], inplace=True)
    
    # Debugging: Check data after SMA calculation
    print(f"Data shape after SMA calculation: {data.shape}")
    if data.empty:
        print("Error: Data is empty after SMA calculation. Adjust SMA windows or data range.")
        return

    # Generate signals with a trend filter
    data['Signal'] = (data['Short_SMA'] > data['Long_SMA']).astype(int)
    data['Shifted_Signal'] = data['Signal'].shift(1)

    # Drop rows with NaN in 'Shifted_Signal'
    data.dropna(subset=['Shifted_Signal'], inplace=True)
    print(f"Data shape after dropping NaNs in Shifted_Signal: {data.shape}")
    if data.empty:
        print("Error: Data is empty after dropping NaNs in Shifted_Signal.")
        return

    # Calculate daily returns
    data['returns'] = data['Close'].pct_change()
    data.dropna(subset=['returns'], inplace=True)
    print(f"Data shape after dropping NaNs in returns: {data.shape}")
    if data.empty:
        print("Error: Data is empty after dropping NaNs in returns.")
        return

    # Calculate Strategy Returns
    data['Strategy_Return'] = data['Shifted_Signal'] * data['returns']

    # Apply stop-loss if specified
    if stop_loss:
        data['Max_Drawdown'] = data['Close'].expanding().max()  # Track maximum price
        data['Drawdown'] = (data['Close'] - data['Max_Drawdown']) / data['Max_Drawdown']
        data.loc[data['Drawdown'] < -stop_loss, 'Strategy_Return'] = 0

    # Ensure there is data left to calculate returns
    if data.empty:
        print("Error: No data available for return calculations.")
        return

    # Calculate total returns
    total_strategy_return = (1 + data['Strategy_Return']).prod() - 1
    total_benchmark_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1

    # Signal Analysis
    buy_signal_count = data['Signal'].sum()
    print(f"\nNumber of Buy Signals: {buy_signal_count}")
    print(f"Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%")

    # Build report
    report = f"""
    Refined Backtest Report for {ticker}
    ==================================
    Period: {start_date} to {end_date}
    SMA Strategy Parameters:
    - Short Window: {short_window} days
    - Long Window: {long_window} days
    - Stop Loss: {stop_loss * 100:.2f}% (if applicable)

    Performance:
    -----------------
    Total Strategy Return: {total_strategy_return * 100:.2f}%
    Total Buy-and-Hold Return (Benchmark): {total_benchmark_return * 100:.2f}%

    Signal Analysis:
    -----------------
    Number of Buy Signals: {buy_signal_count}
    Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%
    """
    print(report)

    # Optional: Return the data for further analysis
    return data

# Example usage
ticker = "TSLA"
data = backtest_sma_strategy(ticker, start_date="2024-07-01", end_date="2025-01-16", short_window=5, long_window=15, stop_loss=0.05)


[*********************100%***********************]  1 of 1 completed

Data shape after SMA calculation: (123, 8)
Data shape after dropping NaNs in Shifted_Signal: (122, 10)
Data shape after dropping NaNs in returns: (121, 11)

Number of Buy Signals: 76
Percentage of Days in Position: 62.81%

    Refined Backtest Report for TSLA
    Period: 2024-07-01 to 2025-01-16
    SMA Strategy Parameters:
    - Short Window: 5 days
    - Long Window: 15 days
    - Stop Loss: 5.00% (if applicable)

    Performance:
    -----------------
    Total Strategy Return: 160.71%
    Total Buy-and-Hold Return (Benchmark): 98.26%

    Signal Analysis:
    -----------------
    Number of Buy Signals: 76
    Percentage of Days in Position: 62.81%
    


In [3]:
# SMA backup
import pandas as pd
import yfinance as yf

def backtest_sma_strategy(ticker, start_date, end_date):
    # Download historical data
    data = yf.download(ticker, start=start_date, end=end_date, group_by='column')
    data.columns = ['_'.join(filter(None, col)).strip() for col in data.columns]
    data.rename(columns=lambda x: x.replace(f"_{ticker}", ""), inplace=True)
    data.reset_index(inplace=True)

    # Ensure 'Close' exists
    if 'Close' not in data.columns:
        print("Error: 'Close' column not found!")
        return

    # Calculate SMAs
    data['9_MA'] = data['Close'].rolling(window=9).mean()
    data['15_MA'] = data['Close'].rolling(window=15).mean()
    data.dropna(subset=['9_MA', '15_MA'], inplace=True)

    # Generate signals
    data['Signal'] = (data['9_MA'] > data['15_MA']).astype(int)
    data['Shifted_Signal'] = data['Signal'].shift(1)

    # Calculate daily returns
    data['returns'] = data['Close'].pct_change()
    data.dropna(subset=['Shifted_Signal', 'returns'], inplace=True)

    # Calculate strategy returns
    data['Strategy_Return'] = data['Shifted_Signal'] * data['returns']

    # Calculate total returns
    total_strategy_return = (1 + data['Strategy_Return']).prod() - 1
    total_benchmark_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1

    # Analyze signal effectiveness
    buy_signal_count = data['Signal'].sum()
    print(f"\nNumber of Buy Signals: {buy_signal_count}")
    print(f"Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%")

    # Build report
    report = f"""
    Backtest Report for {ticker}
    ==================================
    Period: {start_date} to {end_date}
    Total Strategy Return: {total_strategy_return * 100:.2f}%
    Total Buy-and-Hold Return (Benchmark): {total_benchmark_return * 100:.2f}%

    Signal Analysis:
    -----------------
    Number of Buy Signals: {buy_signal_count}
    Percentage of Days in Position: {buy_signal_count / len(data) * 100:.2f}%
    """
    
    print(report)

    # Optional: Return the data for further analysis
    return data

# Example usage
ticker = "SPY"
data = backtest_sma_strategy(ticker, start_date="2024-07-01", end_date="2025-01-16")


[*********************100%***********************]  1 of 1 completed


Number of Buy Signals: 68
Percentage of Days in Position: 55.74%

    Backtest Report for SPY
    Period: 2024-07-01 to 2025-01-16
    Total Strategy Return: -8.52%
    Total Buy-and-Hold Return (Benchmark): 7.73%

    Signal Analysis:
    -----------------
    Number of Buy Signals: 68
    Percentage of Days in Position: 55.74%
    
